In [11]:
import numpy as np

from dataset import SyntheticSlateBanditDataset
from policy import gen_eps_greedy

In [3]:
# settings

random_state=12345
n_rounds=100
dim_context=10
n_actions=15
len_list=3
alpha = np.array([1.,.5,.1])

In [4]:
dataset = SyntheticSlateBanditDataset(
    n_unique_action=n_actions,
    len_list=len_list,
    dim_context=dim_context
)
bandit_feedback = dataset.obtain_batch_bandit_feedback(n_rounds=n_rounds)

In [5]:
evaluation_policy = gen_eps_greedy(expected_reward=bandit_feedback['expected_reward'])

In [6]:
policy_value = dataset.calc_ground_truth_policy_value(
    expected_reward=bandit_feedback['expected_reward'],
    evaluation_policy=evaluation_policy,
    alpha=alpha
)
policy_value

1.0629651155012334

In [10]:
np.mean((evaluation_policy / bandit_feedback["ranking_pi_b"]).max(1))

8.358791399895256e+18